In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import geopandas as gpd

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
url='https://matiascerda.carto.com/api/v2/sql?format=CSV&filename=datos-movilidad-isci.csv&q=select%20*%20from%20data_variacion'

In [5]:
sql="select * from geo.zonas_censales"
zonas= gpd.GeoDataFrame.from_postgis(sql,con=cnx, geom_col='geometry')
zonas['GEOCODIGO']=zonas['GEOCODIGO'].astype(str).str.strip()

In [6]:
zonas

,OBJECTID,REGION,NOM_REGION,PROVINCIA,NOM_PROVINCIA,COMUNA,NOM_COMUNA,URBANO,COD_DISTRITO,COD_ZONA,GEOCODIGO,Shape_Length,Shape_Area,geometry,TOTAL_PERSONAS,PERSONAS_0_A_5_AÑOS,PERSONAS_6_A_14_AÑOS,PERSONAS_15_A_64_AÑOS,PERSONAS_MAS_65_AÑOS
0,1,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,1,3,13404011003,0.030604,0.000043,"POLYGON ((-70.72732 -33.80301, -70.72803 -33.8...",5608.0,430.0,918.0,3776.0,214.0
1,2,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,1,2,13404011002,0.045283,0.000071,"POLYGON ((-70.70792 -33.80101, -70.70805 -33.8...",2435.0,165.0,238.0,1697.0,292.0
2,3,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,4,13404061004,0.084715,0.000102,"POLYGON ((-70.70724 -33.80454, -70.70726 -33.8...",4074.0,239.0,429.0,2869.0,396.0
3,4,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,1,13404061001,0.052736,0.000112,"POLYGON ((-70.74243 -33.81015, -70.74229 -33.8...",4014.0,299.0,519.0,2747.0,403.0
4,5,13,REGIÓN METROPOLITANA DE SANTIAGO,134,MAIPO,13404,PAINE,PAINE,6,2,13404061002,0.024273,0.000035,"POLYGON ((-70.73996 -33.80852, -70.73996 -33.8...",3738.0,221.0,309.0,2572.0,437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4860,50,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,7,6,12101071006,0.085910,0.000389,"POLYGON ((-70.93557 -53.20309, -70.93549 -53.2...",3159.0,280.0,459.0,2134.0,95.0
4861,51,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,1,12101101001,0.071263,0.000042,"POLYGON ((-70.93740 -53.23148, -70.93743 -53.2...",205.0,13.0,25.0,128.0,39.0
4862,52,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,4,12101101004,0.157769,0.000532,"POLYGON ((-70.94820 -53.14980, -70.94825 -53.1...",5280.0,342.0,659.0,3847.0,161.0
4863,53,12,REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA,121,MAGALLANES,12101,PUNTA ARENAS,PUNTA ARENAS,10,3,12101101003,0.049192,0.000072,"POLYGON ((-70.91382 -53.12629, -70.91389 -53.1...",3937.0,167.0,456.0,2880.0,124.0


In [7]:
crs=zonas.crs

In [8]:
#diferencias entre entradas y salidas de zonas censales estimadas por ISCI-entel Ocean
ISCIentel=pd.read_csv(url,encoding='utf-8')
#diferencias entre entradas
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']].astype(int)
#diferencias entre salidas
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel['dif_salida'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']].astype(int)
ISCIentel['zona_censal']=ISCIentel['zona_censal'].astype(str).str.strip()
ISCIentel=ISCIentel.drop(['the_geom','cartodb_id','the_geom_webmercator'],axis=1)
ISCIentel=ISCIentel.merge(zonas,right_on='GEOCODIGO',left_on='zona_censal',how='left')
ISCIentel['date']=pd.to_datetime('2020-'+ISCIentel['week'].astype(str)+'-6',format='%Y-%W-%w')
ISCIentel['salidas']=(ISCIentel['dif_salida_lim_sup']-ISCIentel['dif_salida_lim_inf'])/2+ISCIentel['dif_salida_lim_inf']
ISCIentel['PERSONAS_MOVILIZADAS']=ISCIentel['PERSONAS_15_A_64_AÑOS']+ISCIentel['PERSONAS_15_A_64_AÑOS']/100*ISCIentel['salidas']

<ipython-input-8-99d76d511709>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-8-99d76d511709>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-8-99d76d511709>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['dif_

In [21]:
ISCIentel = gpd.GeoDataFrame(ISCIentel)
ISCIentel.set_crs(crs)

,zona_censal,comuna,region,week,dif_salida,dif_entrada,dif_entrada_lim_inf,dif_entrada_lim_sup,dif_salida_lim_inf,dif_salida_lim_sup,...,Shape_Area,geometry,TOTAL_PERSONAS,PERSONAS_0_A_5_AÑOS,PERSONAS_6_A_14_AÑOS,PERSONAS_15_A_64_AÑOS,PERSONAS_MAS_65_AÑOS,date,salidas,PERSONAS_MOVILIZADAS
0,15101051004,ARICA,REGIÓN DE ARICA Y PARINACOTA,23,"[-21%,+20%,]","[-21%,+20%,]",-21,20,-21,20,...,0.000017,"POLYGON ((-70.28390 -18.47475, -70.28386 -18.4...",2196.0,151.0,264.0,1402.0,279.0,2020-06-13,-0.5,1394.990
1,13126091003,QUINTA NORMAL,REGIÓN METROPOLITANA DE SANTIAGO,44,"[+61%,+100%,]","[+61%,+100%,]",61,100,61,100,...,0.000019,"POLYGON ((-70.70903 -33.40783, -70.70965 -33.4...",2143.0,126.0,208.0,1399.0,331.0,2020-11-07,80.5,2525.195
2,13401031002,SAN BERNARDO,REGIÓN METROPOLITANA DE SANTIAGO,41,"[-21%,+20%,]","[-21%,+20%,]",-21,20,-21,20,...,0.000025,"POLYGON ((-70.68567 -33.58493, -70.68527 -33.5...",3533.0,197.0,324.0,2317.0,533.0,2020-10-17,-0.5,2305.415
3,4106011001,VICUÑA,REGIÓN DE COQUIMBO,29,"[-40%,-21%,]","[-21%,+20%,]",-21,20,-40,-21,...,0.000092,"POLYGON ((-70.72404 -30.02253, -70.72413 -30.0...",3918.0,331.0,540.0,2626.0,355.0,2020-07-25,-30.5,1825.070
4,4301161001,OVALLE,REGIÓN DE COQUIMBO,9,"[-21%,+20%,]","[-21%,+20%,]",-21,20,-21,20,...,0.000198,"POLYGON ((-71.11972 -30.62603, -71.11984 -30.6...",3768.0,254.0,412.0,2435.0,459.0,2020-03-07,-0.5,2422.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209190,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,47,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,0.000019,"POLYGON ((-70.64423 -33.39011, -70.64468 -33.3...",2659.0,136.0,227.0,1853.0,383.0,2020-11-28,-0.5,1843.735
209191,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,48,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,0.000019,"POLYGON ((-70.64423 -33.39011, -70.64468 -33.3...",2659.0,136.0,227.0,1853.0,383.0,2020-12-05,-0.5,1843.735
209192,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,49,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,0.000019,"POLYGON ((-70.64423 -33.39011, -70.64468 -33.3...",2659.0,136.0,227.0,1853.0,383.0,2020-12-12,-0.5,1843.735
209193,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,50,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,0.000019,"POLYGON ((-70.64423 -33.39011, -70.64468 -33.3...",2659.0,136.0,227.0,1853.0,383.0,2020-12-19,-0.5,1843.735


In [22]:
name='ISCIentel'
schema='movilidad'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema movilidad exists, will not be created


In [ ]:
ISCIentel.to_postgis(name=name, schema=schema,con=cnx,if_exists='replace')